<a href="https://colab.research.google.com/github/ajasja/INSRTR/blob/main/INSRTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSRTR -- Designed allosteric protein logic

In [11]:
#@title Setup and import packages
!pip install py3Dmol biopython > /dev/null
import py3Dmol
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets.widgets import IntSlider, FloatSlider, Checkbox, ToggleButtons, HBox, VBox, Button, Text




In [4]:
#@title Upload pdb 
from google.colab import files
up_files = files.upload()
up_file_name = list(up_files.keys())[0] # get the first file


with open(up_file_name, 'r') as file:
    model = file.read()

from Bio.PDB import PDBParser, PPBuilder

#get sequence from PDB
ppb=PPBuilder()
structure = PDBParser(PERMISSIVE=1, QUIET=True).get_structure(up_file_name, up_file_name)
seq = ":".join([str(pp.get_sequence()) for pp in ppb.build_peptides(structure)]) # should in theory support multichain structures. 

Saving TET12SN-test-asp.pdb to TET12SN-test-asp (1).pdb


In [17]:
#@title Run the editor/prediction

p = py3Dmol.view(width=800, height=480)

p.addModels(model, "pdb")

p.setStyle({'cartoon': {'color':'spectrum'}})

p.zoomTo()
p.show()

def on_position_change(change):
    p.addStyle({'sphere':{'hidden':True}});
    p.addStyle({'resi': position.value, 'atom': 'CA'}, {'sphere':{'color': 'red', 'scale':2, 'hidden':False}});
    p.update()

def on_find_pos_click(change):
    print("Find best position not yet implemented")

position=IntSlider(min=1, max=len(seq), value=1, step=1, continuous_update=True, description='position_CC', tooltip='Position of CC insertion')
position.observe(on_position_change, names='value')
on_position_change(position)

find_pos=Button(description='Find best', tooltip='Find best position in the loop')
find_pos.on_click(on_find_pos_click)

CC1 = widgets.Dropdown(
    options=['P7', 'N7', 'N8'],
    value='P7',
    description='CC1:',
    tooltip='Select CC to insert'
)

Link1 = widgets.Dropdown(
    options=['GSSGS', 'SPGGS', 'SGSSGS'],
    value='GSSGS',
    description='Link1:',
    tooltip='Select N-terminal linker for CC'
)

Link2 = widgets.Dropdown(
    options=['GSGGS', 'GGSGGG'],
    value='GSGGS',
    description='Link2:',
    tooltip='Select C-terminal linker for CC'
)

CC2 = widgets.Dropdown(
    options=['N8', 'N7', 'P7'],
    value='N8',
    description='CC2:',
    tooltip='Select CC ligand'
)

display(
    HBox([position, find_pos]),
    HBox([CC1, Link1, Link2]),
    HBox([CC2])
)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [1]:
import pymol

In [ ]:
from IPython.utils import io
import tqdm.notebook
import os
"""The PyMOL installation is done inside two nested context managers. This approach
was inspired by Dr. Christopher Schlicksup's (of the Phenix group at
Lawrence Berkeley National Laboratory) method for installing cctbx
in a Colab Notebook. He presented his work on September 1, 2021 at the IUCr
Crystallographic Computing School. I adapted Chris's approach here. It replaces my first approach
that requires seven steps. My approach was presentated at the SciPy2021 conference
in July 2021 and published in the
[proceedings](http://conference.scipy.org/proceedings/scipy2021/blaine_mooers.html).
The new approach is easier for beginners to use. The old approach is easier to debug
and could be used as a back-up approach.

"""
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(30)

        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install -c schrodinger pymol-bundle --yes

        pbar.update(100)

In [ ]:
%%bash
if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
fi